In [69]:
import requests

In [70]:
URL = "https://www.coindesk.com"

In [71]:
resp = requests.get(URL)

In [74]:
resp

<Response [200]>

In [4]:
import re

### climb market page

In [5]:
def get_list_page(url):
    HOST = "https://www.coindesk.com"
    resp = requests.get(url)
    links = re.findall('<a class="fade" href="(https://www.coindesk.com/.*)" title=".*">.*</a>', resp.text)    
    detail_page_links = [link for link in links]
    return detail_page_links

In [6]:
get_list_page("https://www.coindesk.com/category/markets-news")

['https://www.coindesk.com/overstocks-medici-ventures-invests-in-blockchain-voting-startup/',
 'https://www.coindesk.com/bitcoin-faces-bearish-move-as-prices-drop-towards-15k/',
 'https://www.coindesk.com/weathering-altcoin-shitstorm-investing-next-one/',
 'https://www.coindesk.com/anti-virus-token-polyswarm-seeks-safer-internet-50-million-ico/',
 'https://www.coindesk.com/2018s-challenge-crypto-assets-really-worth/',
 'https://www.coindesk.com/xrp-dichotomy-ripple-price/',
 'https://www.coindesk.com/crypto-didnt-give-us-2017/',
 'https://www.coindesk.com/long-blockchain-to-enter-bitcoin-mining-after-crypto-pivot-claim/',
 'https://www.coindesk.com/ripple-claims-3-money-transfer-firms-xrp-2018/',
 'https://www.coindesk.com/ripple-eyes-retreat-after-record-price-highs/']

In [10]:
!mkdir /home/ubuntu/bb105/coindesk_data/unclean

In [30]:
DATA_PATH = '/home/ubuntu/bb105/coindesk_data/unclean'
def dump_page(url):
    filename = "_".join(url.replace("-","_").split('/')[-3:])
    resp = requests.get(url)
    with open(DATA_PATH + '/' + filename, 'w') as f:
        f.write(resp.text)
    return filename

In [94]:
dump_page("https://www.coindesk.com/category/markets-news")

'www.coindesk.com_category_markets_news'

In [92]:
range(1,10)
from bs4 import BeautifulSoup
urls = "https://www.coindesk.com/category/markets-news/page/{}"
list=[]
for num in range(10): 
    list.extend([urls.format(num)])
list

['https://www.coindesk.com/category/markets-news/page/0',
 'https://www.coindesk.com/category/markets-news/page/1',
 'https://www.coindesk.com/category/markets-news/page/2',
 'https://www.coindesk.com/category/markets-news/page/3',
 'https://www.coindesk.com/category/markets-news/page/4',
 'https://www.coindesk.com/category/markets-news/page/5',
 'https://www.coindesk.com/category/markets-news/page/6',
 'https://www.coindesk.com/category/markets-news/page/7',
 'https://www.coindesk.com/category/markets-news/page/8',
 'https://www.coindesk.com/category/markets-news/page/9']

In [304]:
range(1,10)
urls="https://www.coindesk.com/category/markets-news/page/{}"
total_linksss=[]
import requests
import re
from bs4 import BeautifulSoup
def get_page_link(url):    
    for num in range(1,10): 
        resp = requests.get(urls.format(num))
        links = re.findall('<a class="fade" href="(https://www.coindesk.com/.*)" title=".*">.*</a>', resp.text)  
        for row in links:
            total_linksss.append(row)
    return 'list created, please type total_link'

In [305]:
get_page_link("1")

'list created, please type total_link'

In [161]:
get_list_page("https://www.coindesk.com/category/markets-news")

MissingSchema: Invalid URL '1': No schema supplied. Perhaps you meant http://1?

In [37]:
import requests

In [39]:
r=requests.get("https://www.coindesk.com/category/markets-news")

In [162]:
get_page_link('1')

'list created, please type total_links'

In [91]:
range(1,10)
total_link=[]
import requests
import re
def get_page_link(url):
    a=url+"/"+"page/{}"   
    for num in range(1,10): 
        resp = requests.get(a.format(num))
        links = re.findall('<a class="fade" href="(https://www.coindesk.com/.*)" title=".*">.*</a>', resp.text)  
        for row in links:
            total_link.append(row)
    return 'list created, please type total_link'

In [92]:
get_page_link("https://www.coindesk.com/category/markets-news")

'list created, please type total_link'

In [93]:
total_link

['https://www.coindesk.com/defying-wider-market-downtrend-bitcoin-cash-eyes-3k/',
 'https://www.coindesk.com/korean-official-cryptocurrency-exchange-ban-not-finalized/',
 'https://www.coindesk.com/bitcoin-price-stays-heavy-amid-korean-regulatory-reports/',
 'https://www.coindesk.com/south-korea-reportedly-expands-crackdown-bitcoin-exchanges/',
 'https://www.coindesk.com/kodaks-cryptocurrency-already-presold/',
 'https://www.coindesk.com/warren-buffett-cryptocurrencies-will-come-to-a-bad-ending/',
 'https://www.coindesk.com/down-but-not-out-ripple-bulls-may-regain-the-upper-hand/',
 'https://www.coindesk.com/coinbase-massive-buy-demand-caused-bitcoin-cash-launch-hiccups/',
 'https://www.coindesk.com/goodbye-icos-hello-taos-tokens-will-change-2018/',
 'https://www.coindesk.com/downside-exposed-bitcoin-continues-slide-to-below-14k/',
 'https://www.coindesk.com/beyond-red-tape-path-ahead-token-sales/',
 'https://www.coindesk.com/telegrams-1-2-billion-ico-ambitious-token-sale-yet/',
 'https

In [94]:
DATA_PATH = '/home/ubuntu/bb105/coindesk_data/unclean'
filename_list=[]
def dump_page(url):
    filename = "".join(url.replace("-","_").replace(".","_").split('/')[-2])
    filename_list.append(filename)
    resp = requests.get(url)
    with open(DATA_PATH + '/' + filename, 'w') as f:
        f.write(resp.text)
    return filename

In [95]:
for url in total_link:
    dump_page(url)

In [96]:
from bs4 import BeautifulSoup
import datetime
import bs4
data = {}
res = []
UTC = datetime.timezone(datetime.timedelta(hours=0))
total_data={}
counter = {}
UNCLEAN_DATA_PATH='/home/ubuntu/bb105/coindesk_data/unclean'
CLEAN_DATA_PATH='/home/ubuntu/bb105/coindesk_data/clean'
def get_detail_and_save(filename):
    with open(UNCLEAN_DATA_PATH + "/" + filename) as f:
        html = f.read()
        soup = BeautifulSoup(html, 'lxml')
        content = soup.select_one('div.article-content-container')
        data['title'] = soup.select_one("h3.article-top-title").text.strip()
        data['author'] = soup.select_one("a.article-container-lab-name").text.strip()
        t_str = soup.select_one("span.article-container-left-timestamp").text.strip()
        data['tstamp'] = datetime.datetime.strptime(t_str, "%b %d, %Y at %H:%M UTC").replace(tzinfo=UTC)
        for ch in content.children:
            if isinstance(ch, bs4.element.NavigableString):
                continue
            elif ch.name == 'script':
                continue
            else:
                res.append(ch.text)
        data['content']=''.join(res).strip().lower().replace(':', '').replace('\xa0',' ').replace('.', '').replace(',', '').replace("?",'')
        for word in data['content'].split():
            word = word
            if word in counter:
                counter[word] += 1
            else:
                counter[word] = 1
        import json
        with open(CLEAN_DATA_PATH + '/' + 'Clean_'+ filename , 'w') as f:
            f.write(json.dumps(counter))
        return 'write finish'

In [105]:
from bs4 import BeautifulSoup
import datetime
import bs4
data = {}
res = []
UTC = datetime.timezone(datetime.timedelta(hours=0))
total_data={}
counter = {}
UNCLEAN_DATA_PATH='/home/ubuntu/bb105/coindesk_data/unclean'
CLEAN_DATA_PATH='/home/ubuntu/bb105/coindesk_data/clean'
def get_detail_and_save(filename):
    with open(UNCLEAN_DATA_PATH + "/" + filename) as f:
        html = f.read()
        soup = BeautifulSoup(html, 'lxml')
        content = soup.select_one('div.article-content-container')
        data['title'] = soup.select_one("h3.article-top-title").text.strip()
        data['author'] = soup.select_one("a.article-container-lab-name").text.strip()
        t_str = soup.select_one("span.article-container-left-timestamp").text.strip()
        data['tstamp'] = datetime.datetime.strptime(t_str, "%b %d, %Y at %H:%M UTC").replace(tzinfo=UTC)
        for ch in content.children:
            if isinstance(ch, bs4.element.NavigableString):
                continue
            elif ch.name == 'script':
                continue
            else:
                res.append(ch.text)
        data['content']=''.join(res).strip().lower().replace(':', '').replace('\xa0',' ').replace('.', '').replace(',', '').replace("?",'')
        for word in data['content'].split():
            word = word
            if word in counter:
                counter[word] += 1
            else:
                counter[word] = 1
        import json
        with open(CLEAN_DATA_PATH + '/' + 'Clean_'+ filename , 'w') as f:
            f.write(json.dumps(counter))
        return 'write finish'

In [114]:
import os
fileslist=[]
def get_file_in_folder(filepath):
    from os.path import join
    for f in os.listdir(filepath):
        fullpath=join(filepath,f)
        if os.path.isfile(fullpath):
            fileslist.append(f)
    return 'fileslist get, type "fileslist" to check '

In [115]:
get_file_in_folder(UNCLEAN_DATA_PATH)

'fileslist get, type "fileslist" to check '

In [116]:
fileslist

['no_new_crypto_coinbase_squashes_exchange_listing_rumors',
 'ice_exchange_unit_seeking_to_list_bitcoin_futures_etf',
 'ether_bucks_bearish_trend_to_hold_above_1200',
 'anti_virus_token_polyswarm_seeks_safer_internet_50_million_ico',
 'comes_cryptos_summer_love',
 'ron_paul_fed_led_quantitative_easing_sparked_cryptocurrency_surge',
 'ether_hits_psychological_milestone_of_1000',
 'japan_new_heart_bitcoin',
 'over_900_ether_starts_2018_at_all_time_price_high',
 'coinbase_massive_buy_demand_caused_bitcoin_cash_launch_hiccups',
 'down_but_not_out_ripple_bulls_may_regain_the_upper_hand',
 'low_can_bitcoin_go_charts_hint_11k_might_play',
 'crypto_market_cap_sets_time_high_bitcoin_share_drops_historic_low',
 'weathering_altcoin_shitstorm_investing_next_one',
 'video_two_bubbles_shapeshift_ceo_talks_crypto_asset_prices',
 '2017_saluting_shakespearean_shitcoin_drama',
 'downside_exposed_bitcoin_continues_slide_to_below_14k',
 'off_the_leash_bitcoin_looks_north_after_breaking_16k',
 'long_blockc

In [117]:
for file in fileslist:
    get_detail_and_save(file)

In [107]:
get_detail_and_save('video_two_bubbles_shapeshift_ceo_talks_crypto_asset_prices')

'write finish'

In [43]:
data

{'author': 'Brendan Bernstein',
 'content': 'coinbase announced thursday afternoon that for now it would not be adding any new cryptocurrencies to either its flagship platform or its other digital asset exchange gdaxin a blog post the company which currently allows customers to buy sell deposit or withdraw bitcoin litecoin ethereum and most recently bitcoin cash said it wanted to explain its process for adding new assetsthe post cited last year’s digital asset framework announcement where gdax outlined the criteria it would follow when considering whether to add a new token or coin to its exchange the company then stated“as of the date of this statement we have made no decision to add additional assets to either gdax or coinbase any statement to the contrary is untrue and not authorized by the company” \nthe update follows rumors that the exchange would add ripple’s xrp token and marks the first statement coinbase made on the matterover the last few weeks xrp has been on a run hitting 